In [1]:
import matplotlib.pyplot as plt
from IPython.display import display


In [2]:
import pandas as pd
import datetime
import seaborn as sns
import numpy as np

# Suppression des messages d'erreur liés à des besoins de mise à jour de syntaxe en Python
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

warnings.filterwarnings("ignore")

On va tenter d'ajouter une variable explicative ayant une bonne corrélation avec la variable precip.Pour cela, on va utiliser l'algorithme knn qui va nous dire s'il pleut ou pas.
La nouvelle variable knn qui sera crée prendra en valeur la prédiction de la classe donnée par l'algorithme knn. ( soit il pleut soit il ne pleut pas)


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer




In [4]:
all_data_fname  = 'defi-ia-2022/Train/Train/all_data.csv'
all_data = pd.read_csv(all_data_fname)
X_fname  = 'defi-ia-2022/Train/Train/X.csv'
X = pd.read_csv(X_fname)

In [5]:

Y = all_data['Y']


In [6]:
Y =pd.DataFrame(Y)
Y.reset_index(inplace=True)
Y.drop('index',inplace = True,axis=1)

In [7]:
Y

,Y
0,3.4
1,11.7
2,1.0
3,5.6
4,3.2
...,...
162085,3.2
162086,0.0
162087,4.4
162088,5.4


In [8]:
#X['class'] = np.where(Y['Y'] == 0,1,2)
#Y = X.loc[:,X.columns.str.startswith('precip')].sum(axis=1)
Class =np.where(Y['Y'] == 0,1,2)
Class = pd.DataFrame(Class)
Class.reset_index(inplace=True)
Class.drop('index',inplace=True,axis=1)

In [9]:
Class

,0
0,2
1,2
2,2
3,2
4,2
...,...
162085,2
162086,1
162087,2
162088,2


In [10]:
from sklearn.model_selection import ShuffleSplit,StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

Utilisons KNN

In [11]:

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]


In [12]:
transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)




In [13]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,Y_train)



predictions_knn = knn.predict(X_test)
accuracy_score(Y_test,predictions_knn)


0.551668825960886

In [14]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]


Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(oob_score=True)

X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

random_forest.fit(X_train,Y_train)


RandomForestClassifier(oob_score=True)

In [16]:
predictions_rf = random_forest.predict(X_test)
accuracy_score(Y_test,predictions_rf)

0.7579122709605774

On récupère les features importantes

Utilisons Support Vector Machine (SVM) afin de classifier les jours où il pleut

In [17]:
# predictions

Régression Logisitique

In [18]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]

transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)



In [19]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(random_state=4,class_weight = {1:0.7,2:1})
lg.fit(X_train,Y_train)


LogisticRegression(class_weight={1: 0.7, 2: 1}, random_state=4)

In [20]:
predictions_lg = lg.predict(X_test)
accuracy_score(Y_test,predictions_lg)

0.6417216772574907

Extra Trees Classifier (possède une faible variance comparé à Random Forest)

In [21]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]

transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

In [22]:
from sklearn.ensemble import ExtraTreesClassifier
extree = ExtraTreesClassifier(
    random_state = 4,
    n_estimators =500 )
extree.fit(X_train,Y_train)

ExtraTreesClassifier(n_estimators=500, random_state=4)

In [23]:
predictions_extree = extree.predict(X_test)
accuracy_score(Y_test,predictions_extree)

0.7575832356509759

In [24]:
Y['Y']

0          3.4
1         11.7
2          1.0
3          5.6
4          3.2
          ... 
162085     3.2
162086     0.0
162087     4.4
162088     5.4
162089     1.2
Name: Y, Length: 162090, dtype: float64

Nous allons utilisé un autre modèle pour prédire le cumul de pluie sur les jours où on a prédit qu'il pleuvait

In [25]:
index_pluie =np.where(Y['Y']!=0)
len(index_pluie)

1

In [26]:
X_train_pluie = X.loc[index_pluie]
X_train_pluie

,number_sta_day,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_7,precip_8,precip_9,day,number_sta,season,month,lat,lon,height_sta
0,14066001_0,3.05,2.57,3.38,3.20,3.85,5.19,6.04,4.43,5.10,...,0.0,0.0,0.0,0,14066001,Hiver,0,49.334,-0.431,2.0
1,14066001_1,4.73,4.22,9.45,11.51,11.52,10.79,10.53,9.66,10.26,...,0.0,0.0,0.0,1,14066001,Hiver,0,49.334,-0.431,2.0
2,14066001_10,3.68,4.70,7.78,8.08,6.89,5.54,7.01,7.10,6.41,...,0.0,0.0,0.0,10,14066001,Hiver,0,49.334,-0.431,2.0
3,14066001_100,2.09,2.75,6.80,6.40,6.73,5.62,4.92,5.49,5.41,...,0.0,0.0,0.0,100,14066001,Printemps,3,49.334,-0.431,2.0
4,14066001_101,1.23,1.35,2.08,2.33,2.64,2.48,2.83,1.69,1.96,...,0.0,0.0,0.0,101,14066001,Printemps,3,49.334,-0.431,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162084,95690001_724,4.04,4.65,4.76,5.20,6.94,5.54,4.34,4.48,5.91,...,0.0,0.0,0.0,359,95690001,Hiver,11,49.108,1.831,126.0
162085,95690001_725,8.48,9.21,7.40,7.22,6.82,6.88,7.09,5.77,6.43,...,0.0,0.0,0.0,360,95690001,Hiver,12,49.108,1.831,126.0
162087,95690001_727,6.04,5.15,2.90,2.43,3.93,4.48,4.48,4.06,3.16,...,0.0,0.0,0.0,362,95690001,Hiver,12,49.108,1.831,126.0
162088,95690001_728,1.78,1.26,9.64,10.86,9.73,9.10,10.45,8.57,9.81,...,0.0,0.0,0.0,363,95690001,Hiver,12,49.108,1.831,126.0


In [27]:
Y_train_pluie = Y.loc[index_pluie]



In [28]:
Y_train_pluie = pd.DataFrame(Y_train_pluie)
Y_train_pluie = Y_train_pluie.reset_index()


In [29]:
Y_train_pluie.drop('index',axis=1,inplace=True)

In [30]:
Y_train_pluie

,Y
0,3.4
1,11.7
2,1.0
3,5.6
4,3.2
...,...
95020,2.4
95021,3.2
95022,4.4
95023,5.4


Après avoir prédit les jours où il pleut, on utilise un modèle prédictif afin de déterminer le cumul de pluie par jour sur les jours où on prédit où il pleut

In [31]:
X_train_pluie.reset_index(inplace=True)

In [32]:
X_train_pluie.drop('index',axis=1,inplace=True)

In [33]:
Y_train_pluie

,Y
0,3.4
1,11.7
2,1.0
3,5.6
4,3.2
...,...
95020,2.4
95021,3.2
95022,4.4
95023,5.4


In [34]:
X_train_pluie

,number_sta_day,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_7,precip_8,precip_9,day,number_sta,season,month,lat,lon,height_sta
0,14066001_0,3.05,2.57,3.38,3.20,3.85,5.19,6.04,4.43,5.10,...,0.0,0.0,0.0,0,14066001,Hiver,0,49.334,-0.431,2.0
1,14066001_1,4.73,4.22,9.45,11.51,11.52,10.79,10.53,9.66,10.26,...,0.0,0.0,0.0,1,14066001,Hiver,0,49.334,-0.431,2.0
2,14066001_10,3.68,4.70,7.78,8.08,6.89,5.54,7.01,7.10,6.41,...,0.0,0.0,0.0,10,14066001,Hiver,0,49.334,-0.431,2.0
3,14066001_100,2.09,2.75,6.80,6.40,6.73,5.62,4.92,5.49,5.41,...,0.0,0.0,0.0,100,14066001,Printemps,3,49.334,-0.431,2.0
4,14066001_101,1.23,1.35,2.08,2.33,2.64,2.48,2.83,1.69,1.96,...,0.0,0.0,0.0,101,14066001,Printemps,3,49.334,-0.431,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95020,95690001_724,4.04,4.65,4.76,5.20,6.94,5.54,4.34,4.48,5.91,...,0.0,0.0,0.0,359,95690001,Hiver,11,49.108,1.831,126.0
95021,95690001_725,8.48,9.21,7.40,7.22,6.82,6.88,7.09,5.77,6.43,...,0.0,0.0,0.0,360,95690001,Hiver,12,49.108,1.831,126.0
95022,95690001_727,6.04,5.15,2.90,2.43,3.93,4.48,4.48,4.06,3.16,...,0.0,0.0,0.0,362,95690001,Hiver,12,49.108,1.831,126.0
95023,95690001_728,1.78,1.26,9.64,10.86,9.73,9.10,10.45,8.57,9.81,...,0.0,0.0,0.0,363,95690001,Hiver,12,49.108,1.831,126.0


Quantile regression

In [35]:
split = ShuffleSplit(n_splits=1, test_size=0.1,random_state = 4)
split.get_n_splits(X_train_pluie, Y_train_pluie)
train_index, test_index = next(split.split(X_train_pluie, Y_train_pluie)) 

X_train,X_test = X_train_pluie.loc[train_index], X_train_pluie.loc[test_index]
Y_train, Y_test = Y_train_pluie.loc[train_index], Y_train_pluie.loc[test_index]

In [36]:

transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

In [38]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_pinball_loss, mean_squared_error


all_models = {}
common_params = dict(
    learning_rate=0.05,
    n_estimators=1,
    max_depth=3,
    min_samples_leaf=9,
    min_samples_split=9,
    max_features = 'auto'
)
for alpha in [0.5]:
    gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
    all_models["q %1.2f" % alpha] = gbr.fit(X_train, Y_train)

In [39]:
all_models

{'q 0.50': GradientBoostingRegressor(alpha=0.5, learning_rate=0.05, loss='quantile',
                           max_features='auto', min_samples_leaf=9,
                           min_samples_split=9, n_estimators=1)}

In [40]:
predictions = all_models['q 0.50'].predict(X_test)
quantile_regression_pred = [[i] for i in predictions]
quantile_regression_pred = np.asarray(quantile_regression_pred)
np.mean(np.abs(quantile_regression_pred-Y_test)/(Y_test+1))*100

Y    58.753434
dtype: float64

Prédictions sur données Test

In [41]:
# Lecture des données de l'ensemble test

path = 'defi-ia-2022/Test/Test/X_station_test.csv'


with open(path, encoding="utf8", errors='ignore') as f:
  contents = f.read()
Xtest = pd.read_csv(path)
display(Xtest)
#Xtest['number_sta']=Xtest['number_sta'].astype('category')

# Tri par station puis par datea
#Xtest = Xtest.sort_values(['number_sta','date'])
#Xtest

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22


In [42]:
split_Id = Xtest['Id'].str.split(pat="_", expand = True)
split_Id = split_Id.rename(columns={0: "number_sta_2", 1: "day", 2: "hour"})
Xtest['number_sta_2'] = split_Id['number_sta_2']
Xtest['day'] = split_Id["day"]
Xtest['hour'] = split_Id["hour"]
Xtest['Id'] = split_Id['number_sta_2'] + "_" + split_Id['day']
Xtest = Xtest.drop("number_sta_2",axis=1)
Xtest.fillna(method='bfill',inplace=True)
display(Xtest)

,dd,hu,td,t,ff,precip,month,Id,day,hour
0,221.0,89.3,275.19,278.35,4.90,0.0,12,14047002_277,277,4
1,221.0,89.3,275.19,278.40,4.90,0.0,12,14047002_277,277,5
2,221.0,89.3,275.19,279.01,4.90,0.0,12,14047002_277,277,6
3,221.0,89.3,275.19,279.66,4.90,0.0,12,14047002_277,277,7
4,221.0,89.3,275.19,279.99,4.90,0.0,12,14047002_277,277,8
...,...,...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176,176,19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176,176,20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176,176,21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176,176,22


In [43]:
Xtest['ff_idx'] = 'ff_' + Xtest["hour"].astype(str)
Xtest['t_idx'] = 't_' + Xtest["hour"].astype(str)
Xtest['td_idx'] = 'td_' + Xtest["hour"].astype(str)
Xtest['hu_idx'] = 'hu_' + Xtest["hour"].astype(str)
Xtest['dd_idx'] = 'dd_' + Xtest["hour"].astype(str)
Xtest['precip_idx'] = 'precip_' + Xtest["hour"].astype(str)

In [44]:
ff = Xtest.pivot(index='Id',columns='ff_idx',values='ff')
t = Xtest.pivot(index='Id',columns='t_idx',values='t')
td = Xtest.pivot(index='Id',columns='td_idx',values='td')
hu = Xtest.pivot(index='Id',columns='hu_idx',values='hu')
dd = Xtest.pivot(index='Id',columns='dd_idx',values='dd')
precip = Xtest.pivot(index='Id',columns='precip_idx',values='precip')

In [45]:
X_reshape = pd.concat([ff ,t,td,hu,dd,precip],axis=1)

In [46]:
X_reshape.reset_index(inplace=True)
X_reshape

,Id,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_21,precip_22,precip_23,precip_3,precip_4,precip_5,precip_6,precip_7,precip_8,precip_9
0,14047002_100,5.38,5.38,5.38,5.38,5.38,5.38,5.38,5.380000,5.38,...,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,14047002_223,4.54,4.54,4.54,4.54,4.54,4.54,4.54,4.540000,4.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14047002_236,3.57,3.57,3.57,3.57,3.57,3.57,3.57,3.570000,3.57,...,0.0,0.0,0.6,0.2,0.6,0.0,0.0,0.8,0.0,0.0
3,14047002_256,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.000000,2.00,...,0.4,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14047002_277,NaN,NaN,4.90,4.90,4.90,4.90,4.90,4.900000,4.90,...,2.4,2.0,0.6,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96050,95690001_95,0.13,0.30,2.37,1.82,2.20,2.44,2.32,2.140000,2.21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96051,95690001_96,3.15,3.92,6.27,5.49,6.19,7.46,6.06,4.710000,5.00,...,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96052,95690001_97,1.41,1.76,6.33,8.23,7.77,6.50,4.28,3.520000,2.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96053,95690001_98,6.90,7.09,8.85,8.58,7.42,7.62,9.56,8.250000,7.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0


In [47]:
X = X_reshape

X['day'] = X['Id'].str.rpartition("_")[2]
X['day'] = X['day'].astype('int')
X['day'] = X['day'] % 365

In [48]:
X['number_sta'] = X['Id'].str.rpartition('_')[0]
X['number_sta'] = X['number_sta'].astype('int64')

In [49]:
X['season'] =pd.cut(X['day'],[0,60,151,243,334], labels=['Hiver','Printemps','Ete', 'Automne'],ordered=False)
X['season'] = X['season'].fillna('Hiver')
X['season']

0        Printemps
1              Ete
2              Ete
3          Automne
4          Automne
           ...    
96050    Printemps
96051    Printemps
96052    Printemps
96053    Printemps
96054    Printemps
Name: season, Length: 96055, dtype: category
Categories (4, object): ['Hiver', 'Printemps', 'Ete', 'Automne']

In [50]:
X['month'] = X['day'] / 30
X['month'] = X['month'].astype('int')

In [51]:
X.reset_index(inplace=True)
X.drop('index',axis=1,inplace=True)

In [52]:
X['month'] = X['month'].astype('category')
X['season'] = X['season'].astype('category')
X['day'] = X['day'].astype('category')

In [53]:
X

,Id,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_4,precip_5,precip_6,precip_7,precip_8,precip_9,day,number_sta,season,month
0,14047002_100,5.38,5.38,5.38,5.38,5.38,5.38,5.38,5.380000,5.38,...,0.0,0.0,0.0,0.0,0.0,1.0,100,14047002,Printemps,3
1,14047002_223,4.54,4.54,4.54,4.54,4.54,4.54,4.54,4.540000,4.54,...,0.0,0.0,0.0,0.0,0.0,0.0,223,14047002,Ete,7
2,14047002_236,3.57,3.57,3.57,3.57,3.57,3.57,3.57,3.570000,3.57,...,0.6,0.0,0.0,0.8,0.0,0.0,236,14047002,Ete,7
3,14047002_256,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.000000,2.00,...,0.0,0.0,0.0,0.0,0.0,0.0,256,14047002,Automne,8
4,14047002_277,NaN,NaN,4.90,4.90,4.90,4.90,4.90,4.900000,4.90,...,0.0,0.0,0.0,0.0,0.0,0.0,277,14047002,Automne,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96050,95690001_95,0.13,0.30,2.37,1.82,2.20,2.44,2.32,2.140000,2.21,...,0.0,0.0,0.0,0.0,0.0,0.0,95,95690001,Printemps,3
96051,95690001_96,3.15,3.92,6.27,5.49,6.19,7.46,6.06,4.710000,5.00,...,0.0,0.0,0.0,0.0,0.0,0.0,96,95690001,Printemps,3
96052,95690001_97,1.41,1.76,6.33,8.23,7.77,6.50,4.28,3.520000,2.42,...,0.0,0.0,0.0,0.0,0.0,0.0,97,95690001,Printemps,3
96053,95690001_98,6.90,7.09,8.85,8.58,7.42,7.62,9.56,8.250000,7.16,...,0.0,0.0,0.0,0.2,0.0,0.0,98,95690001,Printemps,3


In [54]:

coords_fname  = 'defi-ia-2022/Other/Other/stations_coordinates.csv'
coords = pd.read_csv(coords_fname)
coords['number_sta'] = coords['number_sta'].astype('category')
display(coords)

,number_sta,lat,lon,height_sta
0,86118001,46.477,0.985,120.0
1,86149001,46.917,0.025,60.0
2,56081003,48.050,-3.660,165.0
3,53215001,47.790,-0.710,63.0
4,22135001,48.550,-3.380,148.0
...,...,...,...,...
320,86137003,47.035,0.098,96.0
321,86165005,46.412,0.841,153.0
322,86273001,46.464,1.042,121.0
323,91200002,48.526,1.993,116.0


In [55]:
X =pd.merge(X,coords,how='left',on='number_sta')
X['number_sta'] = X['number_sta'].astype('category')
X.fillna(method = 'bfill',inplace = True)


In [58]:

X_test = transformer.fit_transform(X)

In [59]:
#predictions_test = random_forest.predict(X_test)
predictions_test_knn = knn.predict(X_test)


ValueError: X has 148 features, but KNeighborsClassifier is expecting 152 features as input.

In [ ]:
predictions_test_rf = random_forest.predict(X_test)
predictions_test_lg = lg.predict(X_test)
predictions_test_extree = extree.predict(X_test)

In [ ]:
predictions_test_knn=pd.DataFrame(predictions_test_knn)
predictions_test_rf=pd.DataFrame(predictions_test_rf)
predictions_test_extree=pd.DataFrame(predictions_test_extree)
predictions_test_lg=pd.DataFrame(predictions_test_lg)

In [ ]:
predictions_test=predictions_test_rf.copy()
predictions_test['knn']=predictions_test_knn[0]
predictions_test['rf']=predictions_test_rf[0]
predictions_test['rf_1']=predictions_test_rf[0]

predictions_test['extree']=predictions_test_extree[0]
predictions_test['lg']=predictions_test_lg[0]
predictions_test.drop([0],axis=1,inplace=True)
predictions_test

In [ ]:
predictions_test.replace({1: 0, 2: 1}, inplace=True)
predictions_test["class"] = predictions_test.product(axis=1)
predictions_test["pluie"]=predictions_test["class"]>0
predictions_test

In [ ]:
index_pluie =np.where(predictions_test["pluie"]==True)

In [ ]:
X_pluie = X.loc[index_pluie]
X_pluie

In [ ]:
X_pluie_test = transformer.fit_transform(X_pluie)

In [ ]:
predictions_test_regressor_quant = all_models['q 0.05'].predict(X_pluie_test)

In [ ]:
tab_reg_quant = {'pred_quant' : predictions_test_regressor_quant ,'Id' : X_pluie['Id']}
tab_reg_quant = pd.DataFrame(tab_reg_quant)
tab_reg_quant

In [ ]:
fname = './defi-ia-2022/Test/Test/Baselines/Baseline_observation_test.csv'
Baseline_observation_test = pd.read_csv(fname)
Baseline_observation_test = Baseline_observation_test.sort_values(by=["Id"])
display(Baseline_observation_test)

In [ ]:
pred = pd.merge(Baseline_observation_test,tab_reg_quant,on ='Id',how='left')
pred

In [ ]:
pred = pred.drop(["Prediction"],axis=1)
pred

In [ ]:
pred.rename(columns = {'pred_quant' : 'Prediction'},inplace=True)
pred['Prediction']=pred['Prediction']+1
pred

In [ ]:
# Enregistrement prédictions

output_file = "prediction_test.csv"
pred.to_csv('./predictions/' + output_file,index=False)